In [1]:
from typing import Dict, List
from pathlib import Path
from tqdm.notebook import trange, tqdm

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)


In [2]:
SEED = 42
DATA = '/mnt/storage/clwork/projects/readvisor/RESPONSE_GENERATION/hospo_respo_datasets/en/en_rrgen.urls.pkl'

In [3]:
# load dataframe
df = pd.read_pickle(DATA)

# add working ID column
df['rrgen_id'] = df.index

print(df['domain'].value_counts())
print(df['source'].value_counts())

# subset for tripadvisor only
df = df[df['source'] == 'tripadvisor']
df = df[df['domain'] == 'Hotel']

print(df['domain'].value_counts())
print(df['source'].value_counts())

# drop unwanted columns
df.drop(columns=['reviewid', 'grpid', 'platformid_rev',
        'url', 'platformrating', 'review_pp', 'review_lang',
        'response_pp', 'response_lang', 'source',
         'sentiment_alpha_system_1', 'response_pp_rg', 'split_imrg_compat'], inplace=True)

# minimum response length (in sentences) is 1!
df = df[df['score:response_sentence_length'] > 1]

df.info()

Hotel         2280886
Restaurant    1046717
Name: domain, dtype: Int64
tripadvisor    3306331
platform         13222
re                8050
Name: source, dtype: Int64
Hotel    2275802
Name: domain, dtype: Int64
tripadvisor    2275802
Name: source, dtype: Int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2177752 entries, 0 to 3327598
Data columns (total 26 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   domain                              string 
 1   rating                              Int64  
 2   review_author                       string 
 3   response_author                     string 
 4   review_clean                        string 
 5   response_clean                      string 
 6   sentiment                           string 
 7   db_internal_id                      Int64  
 8   establishment                       string 
 9   trip_id                             Int64  
 10  trip_url                         

In [12]:
# df.set_index('rrgen_id', inplace=True, verify_integrity=False)

In [15]:
# df.head()

,domain,rating,review_author,response_author,review_clean,response_clean,sentiment,db_internal_id,establishment,trip_id,trip_url,country,split,score:review_response_length_ratio,score:response_sentence_length,score:genericness_semantic_avg,score:genericness_length_ratio,score:review_response_wmd,xml_file_name,city,trip_city_id,trip_review_id,trip_review_url,authoridplatform,authorurlplatform
rrgen_id,,,,,,,,,,,,,,,,,,,,,,,,,
0,Hotel,5,CharlieGirl83,StGilesHouse,"Perfect relaxing city getaway! ---SEP--- In need of some TLC I booked last minute. I called direct to the hotel the lovely receptionist was able to book me a room for that night on a dinner, b&b spa package for 1 - this was advertised on their website for a couple but I thought I'd try my luck solo. Within 20 minutes the receptionist called me back to confirm that I could have my treatment when I arrived at 6pm or the next day at 2pm. I opted for immediate relaxation and asked for 6pm followed by a dinner table for 1 at 7.30pm. On arrival around 4pm the reception was picture perfect and I knew I was in for a treat. They took my bags up to my room and gave me a tour of the dinning area and instructions on how to get to the spa on the basement level. I was allocated room 3 The Manhattan room - AMAZING room more like. Colour scheme was egg shell blue green with a mellow brown - very bright and refreshing modern decor. The ensuite had a huge walkin shower with the biggest shower head I have ever seen the sink unit was also very unique and contemporary with a raised glass top and sunken metal sink. The usual mini bar and snacks were on offer and free tea and coffee making with fox's mints and a large complimentary bottle of sping water. A DVD list as long as my arm was availble for free use just ask at reception. After getting comfy in my room and feeling totally relaxed already I put on my hotel robe and wondered down to the spa in just my robe and socks. The wonderful aroma that met me in the spa was heavenly, the neck, back and shoulder massage was also divine, and the therapist was able to adjust this so I didn't need to lay on my tummy as I was just recovering from surgery. After feeling totally pampered and chilled out I had dinner in the restaurant - it was very quiet, but that suited me. A mix of young couples, a family group and some ladies were gathered in the bar area, some outside in the sun trap tuscan style dinning area. The menu looked very appetising, a lot of fresh fish dishes, a few meat dishes and a couple of vegetarian. I had the mixed seafood platter to start with which was lovely and the mustard lamb for mains. Big portions and amazing flavours. After a few drinks with some local friends it was off to the super cozy bed - I was pleasantly surprised by how quiet the room was being a city centre location. It is away from the main roads and noisy nightlife part of the city! Breakfast the next morning was a continental style buffet with lots of fresh fruit, juices, teas, cereals etc plus the full cooked menu options - I made a special request for poached eggs on toast and a bit of bacon - it hit the spot! The hotel is right in the city centre, no need for a car to get anywhere. Norwich is a brilliant, vibrant city full of culture and friendly people. St Giles House Hotel is perfectly located to explore or if your feeling under the weather like myself just stay and chill in the hotel bar or lounge or ever so comfy fluffy beds. A huge thank you to all the staff that made my stay the perfect one night getaway relaxing pamper break that I needed. They were friendly welcoming and very helpful and accomodated to all individual my needs.","Many thanks for such positive feedback from your recent stay at St Giles House. It is great to see so many positive comments & am pleased the hotel was the ideal setting for your much needed break. Great to see also that you made good use of our Spa facilities at the Hotel, and fully relaxed and got truly pampered. It is nice t

In [2]:
# merge info with existing trip_hotels pickled DF used for experiments
df_500k = pd.read_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.pkl')

if 'trip_review_url' not in df_500k.columns:
    df_reduced = df[['rrgen_id', 'trip_review_url', 'trip_review_id']]
    df_500k = pd.merge(df_500k, df_reduced, on='rrgen_id')
    #df_500k.sample(10)
    #df_500k.to_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.pkl')
else:
    print('trip_review_url found in df columns, so no update required')


trip_review_url found in df columns, so no update required


In [3]:
df_500k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 22 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Unnamed: 0                          500000 non-null  int64  
 1   domain                              500000 non-null  object 
 2   rating                              500000 non-null  int64  
 3   review_author                       499949 non-null  object 
 4   response_author                     495164 non-null  object 
 5   review_clean                        500000 non-null  object 
 6   response_clean                      500000 non-null  object 
 7   sentiment                           500000 non-null  object 
 8   db_internal_id                      500000 non-null  int64  
 9   establishment                       500000 non-null  object 
 10  trip_id                             500000 non-null  int64  
 11  trip_url                  

In [4]:
def generate_url_file(df):

    for split in ['test', 'valid', 'train']:
        rrgen_id_file = f'/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/{split}.rrgen_id'
        trip_url_file = f'/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/{split}.trip_url'
        trip_id_file = f'/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/{split}.trip_id'
        print(split)
        df = df[df['split'] == split]
        print(len(df))
        with open(rrgen_id_file, 'r', encoding='utf8') as rrgen_id_f:
            with open(trip_url_file, 'w', encoding='utf8') as trip_url_f:
                with open(trip_id_file, 'w', encoding='utf8') as trip_id_f:
                    for i, line in tqdm(enumerate(rrgen_id_f)):
                        item_id = int(line.strip())
                        row = df.query('rrgen_id==@item_id')
                        trip_url = row['trip_review_url'].item()
                        trip_id = row['trip_review_id'].item()
                        trip_url_f.write(f'{trip_url}\n')
                        trip_id_f.write(f'{trip_id}\n')
                        
generate_url_file(df_500k)

train
450367
